In [1]:
import pandas as pd
import numpy as np
import re, math
from collections import Counter
from googlemaps import convert
from googlemaps import Client
from googlemaps.convert import as_list

WORD = re.compile(r'\w+')

In [2]:
def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])
     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)
     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [41]:
metadata = pd.read_csv('test2.csv', low_memory=False)
print(metadata.head())
print("Select your preferred category:\n1.Adventureous \n2.historical heritage\n3.Romantic\n4.Relaxation")
text1 = input("Enter User Interests: ")   #user preference
vector1 = text_to_vector(text1)
C = metadata['p_rating'].mean()
m = metadata['count'].quantile(0.75)
def weighted_rating(x, m=m, C=C):
    v = x['count']
    R = x['p_rating']
    # Calculation based on the Bayesian Rating Formula
    return (v/(v+m) * R) + (m/(m+v) * C)


                            category  distance     duration nearby_places  \
0  Adventureous ,Relaxation,Romantic  139.16km  137.7166667          NONE   
1  Adventureous ,Relaxation,Romantic   80.79km  150.6666667          NONE   
2           Adventureous ,Relaxation   81.72km  152.5833333          NONE   
3           Adventureous ,Relaxation   90.51km  179.1666667          NONE   
4                Historical Heritage  133.04km  124.5833333          NONE   

                        title  url  p_rating  count  itemId  
0                   Thandiani  NaN       2.2  158.0     1.0  
1  Dunga Gali Pine Line Track  NaN       4.6  652.0     2.0  
2               Mukshpuri Top  NaN       5.0  783.0     3.0  
3                   Miranjani  NaN       4.9  482.0     4.0  
4          Old Lockhart House  NaN       4.2  890.0     5.0  
Select your preferred category:
1.Adventureous 
2.historical heritage
3.Romantic
4.Relaxation
Enter User Interests: romantic


In [42]:
metadata['category'] = metadata['category'].apply(clean_data)
metadata['score'] = metadata.apply(weighted_rating, axis=1).fillna(0)
cos=[]
for i in list(metadata['category']):
    #print(type(i))
    text2 = i
    vector2 = text_to_vector(text2)
    cosine = get_cosine(vector1, vector2)
    cos.append(cosine)
metadata['cosine']=cos
x=metadata['cosine']>0.0
rec=pd.DataFrame(metadata[x])
rec=rec.sort_values('score',ascending=False)
src=input("Enter your location: ")
dest=list(rec['title'])
print(dest)
    

Enter your location: ss world park
['Dunga Gali Pine Line Track', 'Head Panjnad', 'Green Spot', 'Gulzar-e-Sadiq', 'Chitral Valley', 'Babusar Pass', 'Rumbur valley', 'Bahawalpur Zoo', 'Bumburet Valley', 'Chitral Gol National Park', 'Kalasha Valleys', 'Thandiani', 'Ss World Park', 'Bahawalpur Trade Center', 'Samandar Katha Lake', 'Lal Suhanra National Park', 'Garam Chashma']


Enter your location: ss world park
['Mukshpuri Top', 'Miranjani', 'Dunga Gali Pine Line Track', 'Head Panjnad', 'Green Spot', 'Gulzar-e-Sadiq', 'Chitral Valley', 'Babusar Pass', 'Rumbur valley', 'Bahawalpur Zoo', 'Bumburet Valley', 'Chitral Gol National Park', 'Kalasha Valleys', 'Shandur Pass', 'Noor Mahal', 'Thandiani', 'Ss World Park', 'Bahawalpur Trade Center', 'Samandar Katha Lake', 'Lal Suhanra National Park', 'Garam Chashma', 'Tirich Mir']